## Preparation

In [41]:
import wrds
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
pd.set_option('display.max_columns', None)
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [42]:
db = wrds.Connection(wrds_username = 'rotmanren') # Change to user's username

Loading library list...
Done


In [43]:
# Load all data
startYear = 2009 #!!!!!!!!!!2009 or 2010, maybe 2010?
# Accruals need data from 2009 to obtain changes
endYear = 2022
dataquery1 = f"""
            SELECT a.DATADATE, a.cik, a.GVKEY, a.tic, a.fyear, a.ib, a.spi, a.at, a.dvc, a.act, a.che, a.lct, a.dlc, a.txp, a.dp, a.csho, a.ceq, a.ivao, a.lt, a.ajex, a.dltt, a.ivst, a.pstk, b.cshom, b.prccm
            FROM comp.funda as a
            JOIN comp_na_daily_all.secm as b
            ON a.tic = b.tic AND a.fyear = b.cyear
            WHERE consol = 'C'
            AND popsrc = 'D'
            AND indfmt = 'INDL'
            AND datafmt = 'STD' 
            AND fyear BETWEEN '{startYear}' AND '{endYear}'
            AND b.cmth = 6.0
        """
inforaw = db.raw_sql(dataquery1)
# Fill NA for selected variables 
cols = ['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']
inforaw[cols] = inforaw[cols].fillna(0)
inforaw = inforaw.dropna()
inforaw = inforaw.reset_index()
# Adjust for stock split
inforaw['csho'] = inforaw['csho'] * inforaw['ajex']
inforaw['mrk_eq'] = inforaw['cshom'] * inforaw['prccm']
inforaw


,index,datadate,cik,gvkey,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,txp,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq
0,0,2010-05-31,0000001750,001004,AIR,2009.0,44.628,-4.302,1501.042,0.000,863.429,79.370,325.550,100.833,3.263,38.930,39.484,746.906,2.143,754.692,1.0,336.191,0.000,0.0,38697000.0,16.0500,6.210868e+08
1,1,2011-05-31,0000001750,001004,AIR,2010.0,73.139,-1.536,1703.727,2.983,913.985,57.433,416.010,114.075,0.000,59.296,39.781,835.845,2.443,868.438,1.0,329.802,0.000,0.0,39025000.0,16.7400,6.532785e+08
2,2,2012-05-31,0000001750,001004,AIR,2011.0,67.723,-13.864,2195.653,12.081,1063.272,67.720,473.226,122.865,0.000,80.333,40.273,864.649,18.869,1329.631,1.0,669.489,0.000,0.0,39739000.0,27.0900,1.076530e+09
3,3,2013-05-31,0000001750,001004,AIR,2012.0,55.000,-21.100,2136.900,11.900,1033.700,75.300,389.000,86.400,0.000,108.600,39.382,918.600,16.800,1217.400,1.0,622.200,0.000,0.0,40288000.0,13.4800,5.430822e+08
4,4,2014-05-31,0000001750,001004,AIR,2013.0,72.900,0.000,2199.500,11.800,1116.900,89.200,402.100,69.700,0.000,113.400,39.560,999.500,5.200,1198.800,1.0,564.300,0.000,0.0,39733000.0,21.9800,8.733313e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68879,127164,2022-01-31,0001640147,339965,SNOW,2021.0,-679.948,0.000,6649.698,0.000,4598.643,3852.093,1397.093,25.101,12.709,21.498,312.377,5049.045,1463.963,1600.653,1.0,181.196,2766.364,0.0,296128000.0,241.8000,7.160375e+10
68880,127165,2023-01-31,0001640147,339965,SNOW,2022.0,-796.705,-5.300,7722.322,0.000,4984.690,4007.868,1993.517,27.301,20.003,63.535,323.305,5456.436,1277.893,2253.707,1.0,224.357,3067.966,0.0,318100000.0,139.0600,4.423499e+10
68881,127166,2021-12-31,0001695295,345920,HYFM,2021.0,13.416,-25.187,891.242,0.000,269.384,28.384,88.415,9.461,0.729,14.934,44.618,635.180,0.000,256.062,1.0,158.112,1.777,0.0,39812000.0,59.1100,2.353287e+09
68882,127167,2022-12-31,0001695295,345920,HYFM,2022.0,-285.415,-208.918,573.559,0.000,154.948,21.291,41.605,11.110,0.451,41.527,45.197,349.881,0.000,223.678,1.0,174.960,0.000,0.0,44869000.0,3.4800,1.561441e+08


In [44]:
startYear_an = 2019
endYear_an = 2019
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2019 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.dropna()
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.reset_index()
df_analyst_raw_test_2019

startYear_an = 2022
endYear_an = 2022
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2022 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.dropna()
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.reset_index()
df_analyst_raw_test_2022



,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,0,002S,2022-12-31,2022-06-16,2.22,3.70,1,19927,002S
1,1,004W,2022-12-31,2022-06-16,4.58,3.69,1,20748,004W
2,2,005H,2022-12-31,2022-06-16,-5.86,-1.20,1,20422,005H
3,3,006K,2022-12-31,2022-06-16,-0.05,0.35,1,20130,006K
4,4,006W,2022-12-31,2022-06-16,16.16,17.91,1,18004,006W
...,...,...,...,...,...,...,...,...,...
2828,3523,ZMH,2022-12-31,2022-06-16,6.89,6.74,1,144559,ZMH
2829,3525,ZTO,2022-12-31,2022-06-16,8.36,7.56,1,28423,ZTO
2830,3526,ZUMZ,2023-01-31,2022-06-16,1.08,3.60,1,162988,ZUMZ
2831,3527,ZUO,2023-01-31,2022-06-16,-0.13,-0.16,1,33232,ZUO


In [45]:
# Ignore
# dataquery1 = f"""
#             SELECT *
#             FROM comp_na_daily_all.secm
#             LIMIT 10
#         """
# db.raw_sql(dataquery1)


In [46]:
# # Load analyst data for three years
# startYear_an = 2019
# endYear_an = 2019
# dataquery_one = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '1'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_one)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_one = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_one = df_analyst_one.dropna()
# df_analyst_one = df_analyst_one.reset_index()

# # print(df_analyst_one.head(20))

# dataquery_two = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '2'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_two)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_two = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_two = df_analyst_two.dropna()
# df_analyst_two = df_analyst_two.reset_index()

# # print(df_analyst_two.head(20))

# dataquery_three = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '3'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_three)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_three = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_three = df_analyst_three.dropna()
# df_analyst_three = df_analyst_three.reset_index()

# # print(df_analyst_three.head(20))

In [47]:
# df_analyst_one = pd.merge(inforaw, df_analyst_two, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
# df_analyst_one['Earning'] = df_analyst_one['ib'] - df_analyst_one['spi']
# df_analyst_one['flat_actual'] = df_analyst_one['actual'] * df_analyst_one['csho']
# df_analyst_one

In [48]:
# df_analyst_one = pd.merge(inforaw, df_analyst_three, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
# df_analyst_one.head(30)

In [49]:
# df_analyst_three.info()

## ADV preparation

In [50]:
# A function taking care of all financial values calculation
def  calValues(db, company_list):
    print('Estimated waiting time 4mins, please wait')
    # Prepare dataset for nine models
    # HVZ one year
    df_HVZ_one = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_1'])
    # HVZ two year
    df_HVZ_two = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_2'])
    # HVZ three year
    df_HVZ_three = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_3'])

    # EP one year
    df_EP_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_1'])
    # EP two year
    df_EP_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_2'])
    # EP three year
    df_EP_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_3'])

    # RI one year
    df_RI_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_1'])
    # RI two year
    df_RI_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_2'])
    # RI three year
    df_RI_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_3'])

    X_pred_HVZs_2019 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic', 'mrk_eq'])
    X_pred_HVZs_2022 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic', 'mrk_eq'])

    X_pred_EPs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E'])
    X_pred_RIs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC'])

    Y_pred_one = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_two = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_three = pd.DataFrame(columns=['tic','E_deflat'])
    count = 0
    
    for cik in company_list['cik'].astype(str):
        count += 1
        # accrualsinfo = db.raw_sql(dataquery)
        infocomp = inforaw.loc[(inforaw['cik'] == cik)].reset_index(drop=True)
        infocomp = infocomp.sort_values(by='fyear')
    

        # HVZ model
        infocomp['E'] = infocomp['ib'] - infocomp['spi']
        infocomp['DD'] = (infocomp['dvc'] > 0).astype(int)
        infocomp['NegE'] = (infocomp['E'] < 0).astype(int)
        infocomp['AC'] = (infocomp['act'] - infocomp['che']).diff() - (infocomp['lct'] - infocomp['dlc'] - infocomp['txp']).diff() - infocomp['dp']
        # The HVZ needs to be devided by CSHO prior to be compared with the rest

        # EP model
        infocomp['E_deflat'] = infocomp['E'] / infocomp['csho']
        infocomp['NegE_E'] = infocomp['E_deflat'] * infocomp['NegE']

        # RI model
        infocomp['B'] = infocomp['ceq'] / infocomp['csho']
        infocomp['WC'] = (infocomp['act'] - infocomp['che']) - (infocomp['lct'] - infocomp['dlc'])
        infocomp['NCO'] = (infocomp['at'] - infocomp['act'] - infocomp['ivao']) - (infocomp['lt'] - infocomp['lct'] - infocomp['dltt'])
        infocomp['FIN'] = ((infocomp['ivst'] - infocomp['ivao']) - (infocomp['dltt'] - infocomp['dlc'] - infocomp['pstk'])) 
        infocomp['TACC'] = (infocomp['WC'] + infocomp['NCO'] + infocomp['FIN']) / infocomp['csho']

        # Compile dataset
        # Drop year differnece axillary 
        infocomp = infocomp[infocomp['fyear'] != 2009]
        
        # Q5 2022 prediction
        X_pred_2022 = infocomp.loc[infocomp['fyear'] == 2022].reset_index(drop=True)
        X_pred_2022 = X_pred_2022.dropna()
        X_pred_HVZ_2022 = X_pred_2022[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic', 'mrk_eq']].reset_index(drop=True)
        X_pred_HVZs_2022 = X_pred_HVZs_2022.append(X_pred_HVZ_2022, ignore_index=True)
        # Year range of training
        infocomptrain = infocomp[(infocomp['fyear'] >= 2010) & (infocomp['fyear'] <= 2019)].reset_index(drop=True)
        # Prediction data
        X_pred_2019 = infocomptrain.loc[infocomptrain['fyear'] == 2019].reset_index(drop=True)
        X_pred_2019 = X_pred_2019.dropna()
        X_pred_HVZ_2019 = X_pred_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic', 'mrk_eq']].reset_index(drop=True)
        X_pred_HVZs_2019 = X_pred_HVZs_2019.append(X_pred_HVZ_2019, ignore_index=True)
        if X_pred_2019.empty is not None: 
            # Yearly data
            Y_pred_2020 = infocomp[(infocomp['fyear'] == 2020)].reset_index(drop=True)
            Y_pred_2020 = Y_pred_2020[['tic','E_deflat']]
            Y_pred_one = Y_pred_one.append(Y_pred_2020, ignore_index=True)

            Y_pred_2021 = infocomp[(infocomp['fyear'] == 2021)].reset_index(drop=True)
            Y_pred_2021 = Y_pred_2021[['tic','E_deflat']]
            Y_pred_two = Y_pred_two.append(Y_pred_2021, ignore_index=True)

            Y_pred_2022 = infocomp[(infocomp['fyear'] == 2022)].reset_index(drop=True)
            Y_pred_2022 = Y_pred_2022[['tic','E_deflat']]
            Y_pred_three = Y_pred_three.append(Y_pred_2022, ignore_index=True)
        ## Calculate HVZ Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E']].reset_index(drop=True)
    

        y = y.rename(columns={"E": "E_1"})
        HVZ_one = pd.concat([X, y], axis=1)
        df_HVZ_one = df_HVZ_one.append(HVZ_one, ignore_index=True)
        df_HVZ_one = df_HVZ_one.dropna()

        ## Calculate HVZ Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_2"})
        HVZ_two = pd.concat([X, y], axis=1)
        df_HVZ_two = df_HVZ_two.append(HVZ_two, ignore_index=True)
        df_HVZ_two = df_HVZ_two.dropna()

        ## Calculate HVZ Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_3"})
        HVZ_three = pd.concat([X, y], axis=1)
        df_HVZ_three = df_HVZ_three.append(HVZ_three, ignore_index=True)
        df_HVZ_three = df_HVZ_three.dropna()

        ## Calculate EP Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)

        X_pred_EP = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_EPs = X_pred_EPs.append(X_pred_EP, ignore_index=True)

        y = y.rename(columns={"E_deflat": "E_1"})
        EP_one = pd.concat([X, y], axis=1)
        df_EP_one = df_EP_one.append(EP_one, ignore_index=True)
        df_EP_one = df_EP_one.dropna()

        ## Calculate EP Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        EP_two = pd.concat([X, y], axis=1)
        df_EP_two = df_EP_two.append(EP_two, ignore_index=True)
        df_EP_two = df_EP_two.dropna()

        ## Calculate EP Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        EP_three = pd.concat([X, y], axis=1)
        df_EP_three = df_EP_three.append(EP_three, ignore_index=True)
        df_EP_three = df_EP_three.dropna()

        ## Calculate RI Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)
        
        X_pred_RI = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_RIs = X_pred_RIs.append(X_pred_RI, ignore_index=True)
        
        y = y.rename(columns={"E_deflat": "E_1"})
        RI_one = pd.concat([X, y], axis=1)
        df_RI_one = df_RI_one.append(RI_one, ignore_index=True)
        df_RI_one = df_RI_one.dropna()

        ## Calculate RI Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        RI_two = pd.concat([X, y], axis=1)
        df_RI_two = df_RI_two.append(RI_two, ignore_index=True)
        df_RI_two = df_RI_two.dropna()

        ## Calculate RI Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        RI_three = pd.concat([X, y], axis=1)
        df_RI_three = df_RI_three.append(RI_three, ignore_index=True)
        df_RI_three = df_RI_three.dropna()
    return df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs_2019, X_pred_HVZs_2022, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three
        # infocomptrain


In [51]:
#Compile companylist for companies that EXISTED in EVERY year between startyeartrain to endyeartest to not Nan the value
year = startYear
query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
company_list = db.raw_sql(query_comp) 
for year in np.arange(startYear,endYear):
    query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
    comp_list = db.raw_sql(query_comp) 
    company_list = pd.merge(comp_list,company_list, on= 'cik', how='inner')
company_list = company_list.dropna().reset_index(drop=True)

# Company list obtained, Get company tic for reviewing convenience
query_comp = (
        f"""
        SELECT DISTINCT cik, MIN (tic) AS tic
        FROM comp.funda
        WHERE fyear BETWEEN '{startYear}' AND '{endYear}'
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        group by cik
        """) 
company_tic_list = db.raw_sql(query_comp) 
company_list = pd.merge(company_list, company_tic_list, left_on = 'cik', right_on = 'cik', how='left')
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")# Make sure we have the tic exist in linkdf so we can find analyst data
company_list = pd.merge(company_list, linkdf, left_on = 'tic', right_on = 'TICKER', how='inner')

print(company_list)

             cik   tic   gvkey TICKER
0     0001119774  CGEN    3172   CGEN
1     0001115837   MBT  137433    MBT
2     0000075362  PCAR    8253   PCAR
3     0001395942   KAR  183581    KAR
4     0000871763   MAN    7007    MAN
...          ...   ...     ...    ...
1854  0001289848  HURN  161997   HURN
1855  0001084048    ZD   66716     ZD
1856  0001345105   CPA   21381    CPA
1857  0000882361  APTO   15490   APTO
1858  0001046025  HFWA   66285   HFWA

[1859 rows x 4 columns]


In [52]:
# Data extraction
df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs_2019, X_pred_HVZs_2022, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three = \
    calValues(db, company_list)

Estimated waiting time 4mins, please wait


## Build Model

In [53]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one, 'HVZ_two': df_HVZ_two, 'HVZ_three': df_HVZ_three,
        'EP_one': df_EP_one, 'EP_two': df_EP_two, 'EP_three': df_EP_three,
        'RI_one': df_RI_one, 'RI_two': df_RI_two, 'RI_three': df_RI_three}

for name, df in data.items(): 
    print(name)
    if 'HVZ' in name:
        X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    if 'one' in name:
        y = df['E_1']
    elif 'two' in name:
        y = df['E_2']
    elif 'three' in name:
        y = df['E_3']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8406273549991103
HVZ_two
Adjusted R-squared Score:  0.7651757641499377
HVZ_three
Adjusted R-squared Score:  0.7206803654926941
EP_one
Adjusted R-squared Score:  0.6517038844402605
EP_two
Adjusted R-squared Score:  0.37627104332115535
EP_three
Adjusted R-squared Score:  0.09523489577762567
RI_one
Adjusted R-squared Score:  0.6586419773745986
RI_two
Adjusted R-squared Score:  0.5278529905972255
RI_three
Adjusted R-squared Score:  0.14284848752340384


In [72]:
# 2019 Prediction
# models = {'HVZ_one': models['HVZ_one'], 'EP_one': models['EP_one'], 'RI_one': models['RI_one'],
#           'HVZ_two': models['HVZ_two'], 'EP_two': models['EP_two'], 'RI_two': models['RI_two'],
#           'HVZ_three': models['HVZ_three'], 'EP_three': models['EP_three'], 'RI_three': models['RI_three']}
# Use HVZ_one only, predict 2020 using 2019 data
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
 
    X = add_constant(X)
    y = Y_pred_one.copy()
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_earnings_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    print(y_pred_df.head(20))



HVZ_one
       tic     y_pred Actual_earnings_for_the_predicted_year      bias  \
1355  JBHT   4.983455                               4.983578  0.000006   
414    UNP   8.291144                               8.292235  0.000006   
652    KSU   7.110773                                7.11281  0.000016   
1351  TTMI    0.71851                               0.714892 -0.000035   
1156  OTEX   1.298178                               1.299608   0.00005   
1308   CNX   0.064276                              -0.043368 -0.000072   
758   ASGN   3.911317                               3.916824  0.000091   
937   GRMN   5.059282                               5.179928  0.000095   
700     BR     4.2829                               4.296264  0.000103   
827   IRWD   0.744632                               0.756819  0.000105   
939    IPG   1.970878                               1.956766 -0.000154   
564   CERN   2.203502                               2.216205  0.000166   
395   ANSS   5.037573         

In [73]:
prediction_2019 = pd.merge(y_pred_df, df_analyst_raw_test_2019, how='inner', left_on=['tic'], right_on=['ticker'])
prediction_2019 = prediction_2019.sort_values(by='accuracy', ascending=True)

prediction_2019.head(20)
# Table of 2019 prediction with analyst data 

,tic,y_pred,Actual_earnings_for_the_predicted_year,bias,accuracy,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,JBHT,4.983455,4.983578,0.000006,0.000006,1622,JBHT,2019-12-31,2019-06-20,5.210,5.70,1,5783,JBHT
1,UNP,8.291144,8.292235,0.000006,0.000006,3749,UNP,2019-12-31,2019-06-20,8.380,9.08,1,10867,UNP
2,KSU,7.110773,7.11281,0.000016,0.000016,1631,KSU,2019-12-31,2019-06-20,6.900,6.81,1,6335,KSU
3,TTMI,0.71851,0.714892,-0.000035,0.000035,3740,TTMI,2019-12-31,2019-06-20,1.130,1.16,1,139804,TTMI
4,OTEX,1.298178,1.299608,0.00005,0.00005,1931,OTEX,2019-06-30,2019-06-20,2.760,2.76,1,61870,OTEX
5,CNX,0.064276,-0.043368,-0.000072,0.000072,592,CNX,2019-12-31,2019-06-20,0.600,0.76,1,120093,CNX
6,ASGN,3.911317,3.916824,0.000091,0.000091,971,ASGN,2019-12-31,2019-06-20,4.610,4.65,1,25749,ASGN
7,GRMN,5.059282,5.179928,0.000095,0.000095,1362,GRMN,2019-12-31,2019-06-20,4.450,3.73,1,141459,GRMN
8,IRWD,0.744632,0.756819,0.000105,0.000105,2722,IRWD,2019-12-31,2019-06-20,0.550,-0.01,1,184256,IRWD
9,IPG,1.970878,1.956766,-0.000154,0.000154,2719,IPG,2019-12-31,2019-06-20,1.930,1.90,1,6136,IPG


In [74]:
X_pred_HVZs_2022_select = X_pred_HVZs_2022

In [75]:
# 2022 Prediciton, Total earning prediction
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2022_select[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
 
    X = add_constant(X)
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs_2022_select['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2022['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    # y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])/(X_pred_HVZs_2022_select['mrk_eq']/X_pred_HVZs_2022_select['csho']/1000000)
    # y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df['Total_Earning_Growth'] = y_pred_df['y_pred'] * X_pred_HVZs_2022_select['csho'] - X['E']
    y_pred_df = y_pred_df.sort_values(by='Total_Earning_Growth', ascending=False)
    print(y_pred_df.head(20))

HVZ_one
       tic     y_pred Total_Earning_Growth
279   AMZN    0.10552          2702.731088
148    CME   12.39738          1752.616391
527    ICE   5.816923          1498.659861
585     BA  -2.971731          1406.123246
290    CCL   -3.46042          1295.331368
33     DUK    7.10003          1284.022981
7        D   2.752167           1188.05962
327    BTI    4.98803          1116.343471
558      F   2.426054          1061.693101
1068   BIP   2.883734           914.845853
742    BIO -91.738089            897.38053
619     SO   4.450973           878.109705
983    RTX   4.208764           783.058687
1051   IBM   9.043407           757.159118
752     ED    6.57741           674.980645
731    BHP   8.673565           672.634797
618     PM   6.593737           660.730275
1067   AMT   6.447958           658.091531
54     KMI   1.404323           608.471333
485    BUD   3.619589           601.490794


In [76]:
prediction_2022 = pd.merge(y_pred_df, df_analyst_raw_test_2022, how='inner', left_on=['tic'], right_on=['ticker'])
prediction_2022 = prediction_2022.sort_values(by='Total_Earning_Growth', ascending=False)
prediction_2022.head(20)

,tic,y_pred,Total_Earning_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,AMZN,0.10552,2702.731088,263,AMZN,2022-12-31,2022-06-16,-0.050,0.77,1,64768,AMZN
1,CME,12.39738,1752.616391,155,CME,2022-12-31,2022-06-16,7.980,7.83,1,149070,CME
2,BA,-2.971731,1406.123246,595,BA,2022-12-31,2022-06-16,-11.060,-0.76,1,2285,BA
3,CCL,-3.46042,1295.331368,467,CCL,2022-11-30,2022-06-16,-4.670,-2.47,1,13498,CCL
4,DUK,7.10003,1284.022981,2100,DUK,2022-12-31,2022-06-16,5.270,5.45,1,4093,DUK
5,D,2.752167,1188.05962,501,D,2022-12-31,2022-06-16,4.110,4.12,1,4029,D
6,BTI,4.98803,1116.343471,311,BTI,2022-12-31,2022-06-16,4.573,4.43,1,1932,BTI
7,F,2.426054,1061.693101,1932,F,2022-12-31,2022-06-16,1.880,1.96,1,4839,F
8,BIO,-91.738089,897.38053,296,BIO,2022-12-31,2022-06-16,14.420,14.50,1,2220,BIO
9,SO,4.450973,878.109705,3133,SO,2022-12-31,2022-06-16,3.600,3.55,1,9850,SO


In [77]:
Accuracy_2019 = prediction_2019[['tic', 'accuracy']]
prediction_2022 = pd.merge(prediction_2022, Accuracy_2019, how='inner', left_on=['tic'], right_on=['tic'])
prediction_2022
# With accuracy from 2019 prediction

,tic,y_pred,Total_Earning_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER,accuracy
0,AMZN,0.10552,2702.731088,263,AMZN,2022-12-31,2022-06-16,-0.05,0.77,1,64768,AMZN,0.012507
1,CME,12.39738,1752.616391,155,CME,2022-12-31,2022-06-16,7.98,7.83,1,149070,CME,0.010078
2,BA,-2.971731,1406.123246,595,BA,2022-12-31,2022-06-16,-11.06,-0.76,1,2285,BA,0.082135
3,CCL,-3.46042,1295.331368,467,CCL,2022-11-30,2022-06-16,-4.67,-2.47,1,13498,CCL,0.274186
4,DUK,7.10003,1284.022981,2100,DUK,2022-12-31,2022-06-16,5.27,5.45,1,4093,DUK,0.004918
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,TSLA,3.388601,-1870.467759,3474,TSLA,2022-12-31,2022-06-16,4.07,4.03,1,184996,TSLA,0.034379
635,OXY,12.261691,-2359.206846,3221,OXY,2022-12-31,2022-06-16,9.35,10.50,1,8068,OXY,0.302121
636,PFE,5.714537,-3192.159595,3075,PFE,2022-12-31,2022-06-16,6.58,6.50,1,8530,PFE,0.002802
637,MSFT,8.758493,-7490.611863,2438,MSFT,2022-06-30,2022-06-16,9.21,9.28,1,12141,MSFT,0.007751


In [78]:
# 2022 Prediciton, EPS prediction
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2022_select[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
 
    X = add_constant(X)
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs_2022_select['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2022['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    # y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])/(X_pred_HVZs_2022_select['mrk_eq']/X_pred_HVZs_2022_select['csho']/1000000)
    # y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df['EPS_Prediction_Growth'] = y_pred_df['y_pred'] - X['E'] / X_pred_HVZs_2022_select['csho']
    y_pred_df = y_pred_df.sort_values(by='EPS_Prediction_Growth', ascending=False)
    print(y_pred_df.head(20))

HVZ_one
       tic     y_pred EPS_Prediction_Growth
599   AMEN  431.80368            366.841416
482    CVR  36.200065             38.135883
1122  MSTR -96.651903             30.614268
742    BIO -91.738089             30.321007
271   FSCR  24.073384             29.667451
624   WEBC  91.360906             28.485744
1064  ISIG  20.623205             21.710573
683   TCCO  19.562836             21.098434
1163  MTEX  17.983292             20.398569
820    NBY  17.475425             19.345204
745   PSTV  10.400737             18.825562
672   REED   9.247652             16.875417
1168  DGLY   7.532006             16.221601
89    PFIN   11.75792             12.210956
171    ADD  -4.528483             11.510507
137   DYNT   9.563676              10.97482
531   AHPI   8.569645               9.90522
121    GHC  48.328171              8.772708
861   NSYS   8.930323              8.183389
75    LEDS   7.531772              8.099653


In [79]:
prediction_2022 = pd.merge(y_pred_df, df_analyst_raw_test_2022, how='inner', left_on=['tic'], right_on=['ticker'])
prediction_2022 = prediction_2022.sort_values(by='EPS_Prediction_Growth', ascending=False)
prediction_2022.head(20)

,tic,y_pred,EPS_Prediction_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,BIO,-91.738089,30.321007,296,BIO,2022-12-31,2022-06-16,14.42,14.50,1,2220,BIO
1,DYNT,9.563676,10.97482,1913,DYNT,2022-06-30,2022-06-16,-1.30,-1.20,1,4124,DYNT
2,WHR,-6.72304,8.054737,3508,WHR,2022-12-31,2022-06-16,19.64,24.86,1,11465,WHR
3,PDEX,7.244609,6.172585,2468,PDEX,2022-06-30,2022-06-16,1.02,0.83,1,12458,PDEX
4,STRT,7.779123,5.965349,2673,STRT,2022-06-30,2022-06-16,1.80,2.51,1,31567,STRT
5,ACU,6.794734,5.936905,403,ACU,2022-12-31,2022-06-16,0.82,2.38,1,1104,ACU
6,CME,12.39738,4.882865,155,CME,2022-12-31,2022-06-16,7.98,7.83,1,149070,CME
7,USAP,4.324385,4.624164,3492,USAP,2022-12-31,2022-06-16,-0.90,-0.16,1,31170,USAP
8,JKS,6.088682,4.55573,1259,JKS,2022-12-31,2022-06-16,2.21,3.87,1,184182,JKS
9,FSTR,1.625446,4.456289,1950,FSTR,2022-12-31,2022-06-16,-4.25,0.31,1,4860,FSTR


In [80]:
Accuracy_2019 = prediction_2019[['tic', 'accuracy']]
prediction_2022 = pd.merge(prediction_2022, Accuracy_2019, how='inner', left_on=['tic'], right_on=['tic'])
prediction_2022

,tic,y_pred,EPS_Prediction_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER,accuracy
0,BIO,-91.738089,30.321007,296,BIO,2022-12-31,2022-06-16,14.42,14.50,1,2220,BIO,126.551802
1,DYNT,9.563676,10.97482,1913,DYNT,2022-06-30,2022-06-16,-1.30,-1.20,1,4124,DYNT,2.8688
2,WHR,-6.72304,8.054737,3508,WHR,2022-12-31,2022-06-16,19.64,24.86,1,11465,WHR,0.662716
3,PDEX,7.244609,6.172585,2468,PDEX,2022-06-30,2022-06-16,1.02,0.83,1,12458,PDEX,0.359793
4,ACU,6.794734,5.936905,403,ACU,2022-12-31,2022-06-16,0.82,2.38,1,1104,ACU,0.219195
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,MTD,34.483695,-5.398775,1480,MTD,2022-12-31,2022-06-16,39.65,38.36,1,65772,MTD,0.006042
635,WIRE,33.993356,-5.578915,2883,WIRE,2022-12-31,2022-06-16,36.91,20.91,1,25572,WIRE,0.005306
636,ORLY,29.445113,-5.848297,2918,ORLY,2022-12-31,2022-06-16,33.44,32.97,1,28180,ORLY,0.02265
637,REGN,37.114964,-6.805657,3264,REGN,2022-12-31,2022-06-16,44.98,44.22,1,23812,REGN,0.050245
